# **Róbert Šafár** & **Matúš Totcimak**
## 3. fáza: *Strojové učenie*
#### Dataset 82
#### Podiel práce 50:50
#
#

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from scipy.stats import shapiro
from sklearn.preprocessing import PowerTransformer
import math
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

pd.set_option('display.max_columns', None)

## Načítanie CSV súborov

In [18]:
con = pd.read_csv("dataset82/connections.csv", sep='\t')
proc = pd.read_csv("dataset82/processes.csv", sep='\t')

### Pred spojením `Connections` a `Processes` odstránime duplikátne záznamy.

In [19]:
con = con.loc[~con.duplicated()].reset_index(drop=True).copy()
proc = proc.loc[~proc.duplicated()].reset_index(drop=True).copy()

### Spojenie `Connections` a `Processes`.

In [20]:
df = pd.merge(con, proc, on=['ts', 'imei', 'mwra'])

### Overenie chýbajúcich hodnôt. Z `EDA` už vieme, že žiadne nie sú.

In [21]:
df.isna().sum()[df.isna().sum() > 0]

Series([], dtype: int64)

### Vymažeme atribút `imei`, pretože ho nepovažujeme za atribút, ktorý by určoval hodnotu `mwra`. Je to len identifikátor zariadení. Takisto aj `ts`.

In [22]:
df = df.drop(columns=['ts', 'imei']).copy()

#
### *`Funkcia:`* Vykreslenie Box-plotov pre každý atribút z `df`.

In [23]:
def show_boxplots(df):
    _, ax = plt.subplots(figsize=(17, 7))
    df.boxplot(ax=ax,
                    boxprops=dict(color='#34495E'),
                    flierprops=dict(markeredgecolor='#9B59B6', markersize=5),
                    medianprops=dict(color='#3498DB', linewidth=1.5),
                    capprops=dict(color='black'),
                    whiskerprops=dict(color='black')
                    )
    ax.grid(True, linewidth=0.3)
    plt.xticks(rotation=270)
    plt.title("Boxploty atribútov X_train")
    plt.ylabel("Hodnoty")
    plt.show()

In [24]:
# show_boxplots(df=df)

### *`Funckia:`* Zmena outlierov na hraničné hodnoty.

In [25]:
def change_outliers_iqr(df, exceptions):
    for column in df:
        if column in exceptions:
            continue
        
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
        df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    
    return df

### Zmena outlierov pre všetky atribúty `df` okrem `df['c.updateassist']`.

In [26]:
df = change_outliers_iqr(df=df, exceptions=['c.updateassist'])

### Odstránenie záznamov, kde `df['c.updateassist'] >= 5`. Rovnako ako v `EDA`.

In [27]:
lines_to_remove = df[df['c.updateassist'] > 5].index
df = df.drop(lines_to_remove).copy()

In [28]:
# show_boxplots(df=df)

#
# **3.1 Jednoduchý klasifikátor na základe závislosti v dátach**

#
## *A) Naimplementujte jednoduchý ID3 klasifikátor s hĺbkou min 2 (vrátane root/koreň).*

#
## *B) Vyhodnoťte Váš ID3 klasifikátor pomocou metrík accuracy, precision a recall.*

#
## *C) Zistite či Váš ID3 klasifikátor má overfit.*

#
# **3.2 Trénovanie a vyhodnotenie klasifikátorov strojového učenia**

#
## *A) Na trénovanie využite jeden stromový algoritmus v scikit-learn.*

#
## *B) Porovnajte s jedným iným nestromovým algoritmom v scikit-learn.*

#
## *C) Porovnajte výsledky s ID3 z prvého kroku.*

#
## *D) Vizualizujte natrénované pravidlá minimálne pre jeden Vami vybraný algoritmus.*

#
## *E) Vyhodnoťte natrénované modely pomocou metrík accuracy, precision a recall.*

#
# **3.3 Optimalizácia alias hyperparameter tuning**

#
## *A) Vyskúšajte rôzne nastavenie hyperparametrov (tuning) pre zvolený algoritmus tak, aby ste optimalizovali výkonnosť (bez underfitingu).*

#
## *B) Vyskúšajte kombinácie modelov (ensemble) pre zvolený algoritmus tak, aby ste optimalizovali výkonnosť (bez underfitingu).*

#
## *C) Využite krížovú validáciu (cross validation) na trénovacej množine.*

#
## *D) Dokážte že Váš nastavený najlepší model je bez overfitingu.*

#
# **3.4 Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasifikáciu**

#
## *A) Stratégie riešenia chýbajúcich hodnôt a outlierov.*

#
## *B) Dátová transformácia (scaling, transformer, ...).*

#
## *C) Výber atribútov, výber algoritmov, hyperparameter tuning, ensemble learning.*

#
## *D) Ktorý model je Váš najlepší model pre nasadenie (deployment)?*

#
## *E) Aký je data pipeline pre jeho vybudovanie na základe Vášho datasetu v produkcii?*